<ul class="breadcrumb">
  <li><a href="1.1.Univaraite_Linear_Mixed_Effects_Model.ipynb">Univariate Basics</a></li>
  <li><a href="2.1.Univariate_Linear_Additive_Genetic_Model.ipynb">Univariate Additive Genetic Model</a></li> 
  <li><a href="3.1.Univariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Univariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univaraite Linear Mixed Effects Model with Genomic Data</font> 
</div>


<button type="button" class="btn btn-lg btn-primary">Data</button>

In [ ]:
using DataFrames,CSV,JWAS,JWAS.Datasets

In [2]:
phenofile = Datasets.dataset("testMME","simple.txt")
genofile  = Datasets.dataset("testMME","genotype.txt")
pedfile   = Datasets.dataset("testMME","pedigree.txt");

In [3]:
;cat $phenofile

Animal,Age,y
S1,1,-0.92
D1,2,-1.05
O1,3,-0.92
O3,2,1.2


In [4]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1,0,1,1,1
D1,2,0,2,2,1
O1,1,2,0,1,0
O3,0,0,2,1,1


In [5]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [6]:
phenotype = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [7]:
phenotype

,Animal,Age,y
1,S1,1,-0.92
2,D1,2,-1.05
3,O1,3,-0.92
4,O3,2,1.2


In [8]:
pedigree = get_pedigree(pedfile);

coding pedigree... 100%|████████████████████████████████| Time: 0:00:01

Finished!



calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


In [9]:
residual_variance = 1.0
genetic_variance  = 2.5
genetic_variance_by_marker    = 1.5
genetic_variance_by_polygenic = 1.0;

In [10]:
model = build_model("y = intercept + Age + Animal",residual_variance)
set_covariate(model,"Age")
set_random(model,"Animal",pedigree,genetic_variance_by_polygenic)

In [11]:
add_genotypes(model,genofile,genetic_variance_by_marker,separator=',');

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [12]:
output=runMCMC(model,phenotype,chain_length=5000,printout_frequency=1000,
methods="BayesC",Pi=0.9,estimatePi=true,output_samples_frequency=100);

The prior for marker effects variance was calculated from genetic varaince and π.
Marker effects variance is 
6.575342.


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             1000
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                                               0.9
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.000






Posterior means at iteration: 1000
Residual variance: 0.935126
Polygenic effects covariance matrix 


running MCMC for BayesC...  2%|                         |  ETA: 0:00:40

[1.006]
Marker effects variance: 4.420928
π: 0.508

Posterior means at iteration: 2000
Residual variance: 0.961096
Polygenic effects covariance matrix 
[1.169]
Marker effects variance: 4.521288
π: 0.519

Posterior means at iteration: 3000
Residual variance: 0.97275
Polygenic effects covariance matrix 
[1.171]
Marker effects variance: 4.608176
π: 0.519


running MCMC for BayesC... 20%|█████                    |  ETA: 0:00:05


Posterior means at iteration: 4000
Residual variance: 0.991496
Polygenic effects covariance matrix 
[1.133]
Marker effects variance: 4.605883
π: 0.528

Posterior means at iteration: 5000
Residual variance: 0.983228
Polygenic effects covariance matrix 
[1.137]
Marker effects variance: 4.541583
π: 0.528


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:01


In [13]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of marker effects"
  "MCMC samples for residual variance"
  "MCMC samples for: π"
  "MCMC samples for polygenic effects var-cov parameters"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [14]:
output["Posterior mean of location parameters"]

7×2 Array{Any,2}:
 "1:intercept : intercept"  -1.7551   
 "1:Age : Age"               0.620289 
 "1:Animal : S1"             0.0945518
 "1:Animal : D1"            -0.025154 
 "1:Animal : O1"            -0.047483 
 "1:Animal : O3"             0.352073 
 "1:Animal : O2"             0.0445188

In [15]:
output["Posterior mean of marker effects"]

5×2 Array{Any,2}:
 "x1"  -0.405506
 "x2"  -0.299928
 "x3"   0.161111
 "x4"  -0.269763
 "x5"   0.290194

In [16]:
output["Posterior mean of Pi"]

0.5284630217340219

In [ ]:
using Plots
histogram(output["MCMC samples for: π"])